In [1]:
!apt-get install  -y openslide-tools
!pip install openslide-python pillow 
# !pip install --upgrade pip 


Reading package lists... Done
Building dependency tree       
Reading state information... Done
openslide-tools is already the newest version (3.4.1+dfsg-4).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.


In [2]:
import numpy as np
from PIL import Image
import openslide
import matplotlib.pyplot as plt
import random
import pandas as pd
import os

import time 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel


In [3]:
def percentage(mask):
    return (np.sum(mask > 0) / mask.size) * 100
def extract_patches(im_slide, ms_slide, level, size_mask , num_patches_needed):
    f = int(ms_slide.level_downsamples[level])
    size_scale = im_slide.level_dimensions[level][0] // ms_slide.level_dimensions[level][0]
    coord_scale = im_slide.level_dimensions[0][0] // ms_slide.level_dimensions[level][0]
    size_image = (size_mask[0] * size_scale, size_mask[1] * size_scale)
    
    ms_width, ms_height = ms_slide.level_dimensions[level]
    
    image_patches , l= [], []
    for y_ms in range(0, ms_height, size_mask[1]):
        for x_ms in range(0, ms_width, size_mask[0]):
            l.append([x_ms , y_ms])
    
    count,used_indices =0, []

    while count < num_patches_needed:
        index = random.randint(0, len(l) - 1)
        if index not in used_indices:
            used_indices.append(index)
            x_ms, y_ms = l[index][0] , l[index][1]
            x_im, y_im = x_ms * coord_scale, y_ms * coord_scale
            mask_patch = ms_slide.read_region((x_ms * f, y_ms * f), level, size_mask).convert("L")
            image_patch = im_slide.read_region((x_im, y_im), level, size_image).convert("RGB")
   
            if (percentage(np.array(mask_patch))) > 30:
                image_patches.append(np.array(image_patch))
                count+=1
                if count == num_patches_needed:
                    break
        else:
            continue
    return  image_patches

In [4]:
# path = "/kaggle/input/datasetfour/mask/case_radboud_0000_tissue.tif"
# path1 = "/kaggle/input/datasetfour/image/case_radboud_0000.tif"

# im_slide = openslide.OpenSlide(path1)
# ms_slide = openslide.OpenSlide(path)

# level = 1
# size_mask = (128,128)
# num_patches_needed=5
# im= extract_patches(im_slide, ms_slide, level, size_mask , num_patches_needed)
# print(np.shape(im))


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


# **DATASET**

In [6]:
class PatchData(Dataset):
    def __init__(self, images, masks, csv_file, num_patches_per_image, level, size_mask):
        self.images_dir = images
        self.masks_dir = masks
        self.num_patches_per_image = num_patches_per_image
        self.level = level
        self.size_mask = size_mask
        
        self.image_list = sorted(os.listdir(self.images_dir))
        self.mask_list = sorted(os.listdir(self.masks_dir))
        
        self.df = pd.read_csv(csv_file)
        
    def __getitem__(self, index):
        # Fetch the image and mask for the current index
        image_file = self.image_list[index]
        impath = os.path.join(self.images_dir, image_file)
        mspath = os.path.join(self.masks_dir, image_file.replace('.tif', '_tissue.tif'))
        
        im_slide = openslide.OpenSlide(impath)
        ms_slide = openslide.OpenSlide(mspath)
        
        # Extract patches from the image
        image_patches = extract_patches(im_slide, ms_slide, self.level, self.size_mask, self.num_patches_per_image)
        
        # Find corresponding data in CSV
        case_id_to_find = image_file  # Keep the .tif extension
        filtered_row = self.df.loc[self.df['case_id'] == case_id_to_find].iloc[0]
        event = filtered_row["event"]
        years = filtered_row["follow_up_years"]
        
        # Create labels for each patch
        image_labels = [[event, years]] * self.num_patches_per_image
        
        # Convert to tensors
        patches_x = torch.tensor(np.array(image_patches), dtype=torch.float32)
        data_y = torch.tensor(np.array(image_labels), dtype=torch.float32)
        
        return patches_x, data_y
    
    def __len__(self):
        # Number of images
        return len(self.image_list)

   

In [7]:
 
    
# images = "/kaggle/input/datasetfour/image"
# masks = "/kaggle/input/datasetfour/mask"
# csv_file = "/kaggle/input/datasetfour/training_labels.csv"
# dataset = PatchData(images=images, 
#                     masks=masks, 
#                     csv_file=csv_file, 
#                     num_patches_per_image=5,  # Number of patches per image
#                     level=1, 
#                     size_mask=(128, 128))
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# # Check shapes
# for x, y in dataloader:
#     print("Training shapes:")
#     print(f"Input shape: {x.shape}")
#     print(f"Label shape: {y.shape}")

> # **MODEL** 

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):


        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.inc = DoubleConv(in_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        self.down4 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(512, 1024))

        self.up1 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.up_conv1 = DoubleConv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.up_conv2 = DoubleConv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.up_conv3 = DoubleConv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.up_conv4 = DoubleConv(128, 64)

        self.outc = nn.Conv2d(64, out_channels, 1)

        # Global Average Pooling and Fully Connected layers for patch-level predictions
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 2)  # 2 outputs: event and years

    def forward(self, x):
        # x shape: [batch_size, num_patches, height, width, channels]
        batch_size, num_patches, height, width, channels = x.shape
        x = x.view(batch_size * num_patches, channels, height, width)

        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5)
        x = self.up_conv1(torch.cat([x4, x], dim=1))
        x = self.up2(x)
        x = self.up_conv2(torch.cat([x3, x], dim=1))
        x = self.up3(x)
        x = self.up_conv3(torch.cat([x2, x], dim=1))
        x = self.up4(x)
        x = self.up_conv4(torch.cat([x1, x], dim=1))

        # Global Average Pooling and Fully Connected layers
        x = self.gap(x)
        x = x.view(batch_size * num_patches, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # Reshape output to match the input shape
        x = x.view(batch_size, num_patches, 2)

        return x

In [9]:
# device = xm.xla_device()
# model = UNet(in_channels=3, out_channels=2).to(device)

# # You can uncomment these lines if you want to test the model with random input
# batch_size = 2
# num_patches = 6
# channels = 3
# height = 1024
# width = 1024
# train_input = torch.randn(batch_size, num_patches, height, width, channels).to(device)
# print(f"Training input shape: {train_input.shape}")
# train_output = model(train_input)
# print(f"Training output shape: {train_output.shape}")

# # Calculate total parameters
# total_params = sum(p.numel() for p in model.parameters())
# print(f"\nTotal number of parameters: {total_params}")

In [10]:
model = UNet(in_channels=3, out_channels=2)

# Use DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = DataParallel(model)

model = model.to(device)

# Dataset and DataLoader setup
images = "/kaggle/input/dddddddd/images"
masks = "/kaggle/input/dddddddd/masks"
csv_file = "/kaggle/input/dddddddd/training_labels.csv"
dataset = PatchData(images=images, 
                    masks=masks, 
                    csv_file=csv_file, 
                    num_patches_per_image=3,  
                    level=1, 
                    size_mask=(64, 64))
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)

criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=1e-2)

# Create the scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=0.1, patience=1)



Using 2 GPUs


In [11]:
def epochnum(file):
    start,end,s = 0,0,file
    for i in range(len(s)):
        if s[i] == "h":
            start = i + 1
            break
    for i in range(len(s)):
        if s[i] == "-":
            end = i
            break
    return int(file[start:end])

In [12]:
def train(model, optim, criterion, dataloader, epochs, device, start_epoch, scheduler):
    model.train()

    for epoch in range(start_epoch + 1, start_epoch + epochs + 1):
        losses = 0
        c = 0
        for img, label in dataloader:
            img, label = img.to(device), label.to(device)
            optim.zero_grad()
            out = model(img)

            loss = criterion(out, label)
            loss.backward()
            optim.step()
            losses += loss.item()
            c += 1
        
        mean_loss = losses / c
        scheduler.step(mean_loss)
        
        print(f"epoch {epoch}, mean_loss {mean_loss:.4f} , total_loss {losses:.4f} ")
        
        if torch.cuda.device_count() == 1 or (torch.cuda.device_count() > 1 and torch.cuda.current_device() == 0):
            model_filename = f"epoch{epoch}-loss{mean_loss:.4f}.pth"
            model_path = os.path.join( "/kaggle/working", model_filename)
            torch.save(model.state_dict(), model_path)
            print(f"Model saved: {model_path}")


In [13]:
path = "/kaggle/working"
pth_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.pth')]
pth_files.sort(key=lambda x: epochnum(os.path.join(path, x)), reverse=True)

if pth_files:
    state = pth_files[0]
    weights = os.path.join(path, state)
    print(f"Resuming from checkpoint: {state}")
    print(f"Loading weights from: {state}")
    model.load_state_dict(torch.load(weights, map_location='cpu'))
    model.to(device)
    
    start_epoch = epochnum(state)
else:
    print("No checkpoint found. Starting from scratch.")
    start_epoch = 0

print(f"Starting from epoch: {start_epoch}")

train(model, optim, criterion, dataloader, epochs=2, device=device, start_epoch=start_epoch, scheduler=scheduler)


No checkpoint found. Starting from scratch.
Starting from epoch: 0


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_34/2476886095.py", line 67, in forward
    x = self.up_conv4(torch.cat([x1, x], dim=1))
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_34/2476886095.py

In [17]:
path = "/kaggle/working"
pth_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.pth')]
pth_files.sort(key=lambda x: epochnum(os.path.join(path, x)) , reverse=True)
for file in pth_files[1:]:
    os.remove(os.path.join(path , file))
    print(f"removed {file}")

removed epoch19-loss0.2804.pth


In [ ]:
pth_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.pth')]
print(pth_files)

In [26]:
# !nvidia-smi
torch.cuda.empty_cache()

In [ ]:
!rm -rf /kaggle/working/*


In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(a.shape , b.shape)

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'im' is a list or array where im[0] is an image
# Adjust the figsize to your desired size (width, height in inches)
plt.figure(figsize=(10, 10))
plt.imshow(im[0])
plt.axis('off') 
plt.show()

In [ ]:
mask_patches, image_patches = extract_patches(im_slide, ms_slide, level, size_mask)

print(f"Number of mask patches: {len(mask_patches)}")
print(f"Number of image patches: {len(image_patches)}")

# Stitch patches
stitched_mask = stitch_patches(mask_patches, size_mask, ms_slide.level_dimensions[level] , "L")
size_scale = im_slide.level_dimensions[level][0] // ms_slide.level_dimensions[level][0]

size_image = (size_mask[0] * size_scale, size_mask[1] * size_scale)
stitched_image = stitch_patches(image_patches, size_image, im_slide.level_dimensions[level]  , "RGB")
print(size_image)
# Display stitched images
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(stitched_mask, cmap='gray')
plt.title('Stitched Mask')
plt.subplot(1, 2, 2)
plt.imshow(stitched_image)
plt.title('Stitched Image')
plt.show()

# Verify dimensions
print(f"Stitched mask dimensions: {stitched_mask.size}")
print(f"Stitched image dimensions: {stitched_image.size}")
print(f"Original mask dimensions at level {level}: {ms_slide.level_dimensions[level]}")
print(f"Original image dimensions at level {level}: {im_slide.level_dimensions[level]}")

# Display a sample patch pair
sample_index = 0  # You can change this to view different patches
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(mask_patches[sample_index], cmap='gray')
plt.title('Sample Mask Patch')
plt.subplot(1, 2, 2)
plt.imshow(image_patches[sample_index])
plt.title('Corresponding Image Patch')
plt.show()

In [ ]:
print(np.shape(image_patches))
print(type(image_patches))

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
def stitch_patches(patches, patch_size, level_dims,mode):
    stitched_image = Image.new(mode, level_dims)
    x_offset = 0
    y_offset = 0
    for patch in patches:
        stitched_image.paste(Image.fromarray(patch), (x_offset, y_offset) )
        x_offset += patch_size[0]
        if x_offset >= level_dims[0]:
            x_offset = 0
            y_offset += patch_size[1]
    return stitched_image

In [ ]:
import numpy as np
from skimage import measure
from scipy import ndimage

def extract_mask_features(mask):
    
    mask = (mask > 0).astype(int)
    features = {}
    
    features['total_pixels'] = mask.size
    features['positive_pixels'] = np.sum(mask)
    features['negative_pixels'] = features['total_pixels'] - features['positive_pixels']
    features['positive_percentage'] = features['positive_pixels'] / features['total_pixels'] * 100

    labeled_mask, num_components = ndimage.label(mask)
    features['num_components'] = num_components

    regions = measure.regionprops(labeled_mask)
    
    if regions:
        areas = [region.area for region in regions]
        features['mean_component_area'] = np.mean(areas)
        features['max_component_area'] = np.max(areas)
        features['min_component_area'] = np.min(areas)
        
        perimeters = [region.perimeter for region in regions]
        features['mean_component_perimeter'] = np.mean(perimeters)
        features['max_component_perimeter'] = np.max(perimeters)
        features['min_component_perimeter'] = np.min(perimeters)
        
        eccentricities = [region.eccentricity for region in regions]
        features['mean_eccentricity'] = np.mean(eccentricities)
        
        solidities = [region.solidity for region in regions]
        features['mean_solidity'] = np.mean(solidities)

    edges = ndimage.sobel(mask)
    features['edge_pixels'] = np.sum(edges > 0)
    features['edge_to_area_ratio'] = features['edge_pixels'] / features['positive_pixels'] if features['positive_pixels'] > 0 else 0

    return features

mask = np.array(slide.read_region((0,0), level, size).convert("L"))
features = extract_mask_features(mask)
print(features)

In [ ]:
import numpy as np
from skimage import measure
from scipy import ndimage

mask = np.array([
    [0, 1, 1, 0, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1],
    [0,0, 0, 1, 1]
])

print("Original mask:")
print(mask)

In [ ]:
labeled_mask, num_components = ndimage.label(mask)
print("\nLabeled mask:")
print(labeled_mask)
print(f"Number of components: {num_components}")

regions = measure.regionprops(labeled_mask)


In [ ]:
areas = [region.area for region in regions]
print("\nAreas:", areas)
print("Mean area:", np.mean(areas))
print("Max area:", np.max(areas))
print("Min area:", np.min(areas))

In [ ]:
perimeters = [region.perimeter for region in regions]
print("\nPerimeters:", perimeters)
print("Mean perimeter:", np.mean(perimeters))
print("Max perimeter:", np.max(perimeters))
print("Min perimeter:", np.min(perimeters))

In [ ]:
eccentricities = [region.eccentricity for region in regions]
print("\nEccentricities:", eccentricities)
print("Mean eccentricity:", np.mean(eccentricities))

In [ ]:
solidities = [region.solidity for region in regions]
print("\nSolidities:", solidities)
print("Mean solidity:", np.mean(solidities))

In [ ]:
edges = ndimage.sobel(mask)
print(edges)
edge_mask = edges > 0
print("\nEdge mask:")
print(edge_mask)
print(mask)
print("Edge pixels:", np.sum(edge_mask))
print("Edge to area ratio:", np.sum(edge_mask) / np.sum(mask))